In [7]:
import os
os.chdir("../")

In [13]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [36]:
df = pd.read_csv("Artificats/Time_Estimation_Dataset/train.csv")

In [37]:
df.head()

,ID,Delivery_person_Age,Delivery_person_Ratings,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,City,Time_taken
0,0x4607,37.0,4.9,22.765049,75.912471,2022-03-19,11:30:00,Sunny,High,2,motorcycle,0.0,Urban,24
1,0xb379,34.0,4.5,13.043041,77.813237,2022-03-25,19:45:00,Stormy,Jam,2,scooter,1.0,Metropolitian,33
2,0x5d6d,23.0,4.4,12.924264,77.688400,2022-03-19,08:30:00,Sandstorms,Low,0,motorcycle,1.0,Urban,26
3,0x7a6a,38.0,4.7,11.053669,77.026494,2022-04-05,18:00:00,Sunny,Medium,0,motorcycle,1.0,Metropolitian,21
4,0x70a2,32.0,4.6,13.012793,80.289982,2022-03-26,13:30:00,Cloudy,High,1,scooter,1.0,Metropolitian,30


In [38]:
df.drop(['ID'], axis=1, inplace=True)

In [39]:
df['Order_Date']=pd.to_datetime(df['Order_Date'])

# Creating three column for day,month and year
df['Order_day']=df['Order_Date'].dt.day
df['Order_month']=df['Order_Date'].dt.month
df['Order_year']=df['Order_Date'].dt.year

In [40]:
df['Time_Orderd'] = pd.to_datetime(df['Time_Orderd'])

# Creating two new column for hour and minute
df['Hour_order']=df['Time_Orderd'].dt.hour
df['Min_order']=df['Time_Orderd'].dt.minute

C:\Users\rahul\AppData\Local\Temp\ipykernel_16200\1731031211.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time_Orderd'] = pd.to_datetime(df['Time_Orderd'])


In [41]:
df

,Delivery_person_Age,Delivery_person_Ratings,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,City,Time_taken,Order_day,Order_month,Order_year,Hour_order,Min_order
0,37.0,4.9,22.765049,75.912471,2022-03-19,2025-01-17 11:30:00,Sunny,High,2,motorcycle,0.0,Urban,24,19,3,2022,11.0,30.0
1,34.0,4.5,13.043041,77.813237,2022-03-25,2025-01-17 19:45:00,Stormy,Jam,2,scooter,1.0,Metropolitian,33,25,3,2022,19.0,45.0
2,23.0,4.4,12.924264,77.688400,2022-03-19,2025-01-17 08:30:00,Sandstorms,Low,0,motorcycle,1.0,Urban,26,19,3,2022,8.0,30.0
3,38.0,4.7,11.053669,77.026494,2022-04-05,2025-01-17 18:00:00,Sunny,Medium,0,motorcycle,1.0,Metropolitian,21,5,4,2022,18.0,0.0
4,32.0,4.6,13.012793,80.289982,2022-03-26,2025-01-17 13:30:00,Cloudy,High,1,scooter,1.0,Metropolitian,30,26,3,2022,13.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,30.0,4.8,26.912328,75.804257,2022-03-24,2025-01-17 11:35:00,Windy,High,1,motorcycle,0.0,Metropolitian,32,24,3,2022,11.0,35.0
45589,21.0,4.6,0.070000,0.070000,2022-02-16,2025-01-17 19:55:00,Windy,Jam,0,motorcycle,1.0,Metropolitian,36,16,2,2022,19.0,55.0
45590,30.0,4.9,13.052394,80.272439,2022-03-11,2025-01-17 23:50:00,Cloudy,Low,1,scooter,0.0,Metropolitian,16,11,3,2022,23.0,50.0
45591,20.0,4.7,11.041753,77.026241,2022-03-07,2025-01-17 13:35:00,Cloudy,High,0,motorcycle,1.0,Metropolitian,26,7,3,2022,13.0,35.0


In [42]:
df.drop(["Time_Orderd", "Order_Date"],axis = 1, inplace= True)

In [43]:
df.shape

(45593, 16)

In [44]:
column = df.pop("Time_taken")
df.insert(15, column.name, column)

In [45]:
def create_delivery_features(df):

    df_features = df
    
    df_features['avg_delivery_time_area'] = df_features.groupby('City')['Time_taken'].transform('mean')

    df_features['traffic_weather_impact'] = df_features.groupby(
        ['Road_traffic_density', 'Weatherconditions']
    )['Time_taken'].transform('mean')

    df_features['vehicle_capacity_utilization'] = (
        df_features['multiple_deliveries'] / 
        df_features.groupby('Type_of_vehicle')['multiple_deliveries'].transform('max')
    ).fillna(0)
    
    return df_features
    
df = create_delivery_features(df)

In [46]:
col = df.pop("Time_taken")
df.insert(18, col.name, col)

In [47]:
df1 = df.copy()

In [48]:
df

,Delivery_person_Age,Delivery_person_Ratings,Delivery_location_latitude,Delivery_location_longitude,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,City,Order_day,Order_month,Order_year,Hour_order,Min_order,avg_delivery_time_area,traffic_weather_impact,vehicle_capacity_utilization,Time_taken
0,37.0,4.9,22.765049,75.912471,Sunny,High,2,motorcycle,0.0,Urban,19,3,2022,11.0,30.0,22.984017,23.448980,0.000000,24
1,34.0,4.5,13.043041,77.813237,Stormy,Jam,2,scooter,1.0,Metropolitian,25,3,2022,19.0,45.0,27.315226,29.850194,0.333333,33
2,23.0,4.4,12.924264,77.688400,Sandstorms,Low,0,motorcycle,1.0,Urban,19,3,2022,8.0,30.0,22.984017,20.297049,0.333333,26
3,38.0,4.7,11.053669,77.026494,Sunny,Medium,0,motorcycle,1.0,Metropolitian,5,4,2022,18.0,0.0,27.315226,20.195518,0.333333,21
4,32.0,4.6,13.012793,80.289982,Cloudy,High,1,scooter,1.0,Metropolitian,26,3,2022,13.0,30.0,27.315226,28.940860,0.333333,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,30.0,4.8,26.912328,75.804257,Windy,High,1,motorcycle,0.0,Metropolitian,24,3,2022,11.0,35.0,27.315226,26.972789,0.000000,32
45589,21.0,4.6,0.070000,0.070000,Windy,Jam,0,motorcycle,1.0,Metropolitian,16,2,2022,19.0,55.0,27.315226,30.219056,0.333333,36
45590,30.0,4.9,13.052394,80.272439,Cloudy,Low,1,scooter,0.0,Metropolitian,11,3,2022,23.0,50.0,27.315226,22.208445,0.000000,16
45591,20.0,4.7,11.041753,77.026241,Cloudy,High,0,motorcycle,1.0,Metropolitian,7,3,2022,13.0,35.0,27.315226,28.940860,0.333333,26


In [49]:
df.isna().sum()

Delivery_person_Age             1854
Delivery_person_Ratings         1908
Delivery_location_latitude         0
Delivery_location_longitude        0
Weatherconditions                616
Road_traffic_density             601
Vehicle_condition                  0
Type_of_vehicle                    0
multiple_deliveries              993
City                            1200
Order_day                          0
Order_month                        0
Order_year                         0
Hour_order                      1731
Min_order                       1731
avg_delivery_time_area          1200
traffic_weather_impact           616
vehicle_capacity_utilization       0
Time_taken                         0
dtype: int64

In [50]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [51]:
mean_features = ["Delivery_person_Age","Delivery_person_Ratings","avg_delivery_time_area"]
mode_features = ["Weatherconditions","multiple_deliveries","traffic_weather_impact",
    "Road_traffic_density","City","Order_day","Order_month","Order_year","Hour_order","Min_order",]
OE_features = ["Vehicle_condition",'Weatherconditions', 'Type_of_vehicle',"Road_traffic_density"]
OHE_features = ["City"]


In [52]:
SI_mean = SimpleImputer(strategy="mean")
SI_mode = SimpleImputer(strategy="most_frequent")
OE = OrdinalEncoder()
OHE = OneHotEncoder()

preprocessor = ColumnTransformer(
    (
        ("mean_features", SI_mean, mean_features),
        ("mode_features", SI_mode, mode_features),
        ("OE_features", OE, OE_features),
        ("OHE_features", OHE, OHE_features)
    )
)

In [53]:
# 1. Handle mean and mode imputation
df[mean_features] = SI_mean.fit_transform(df[mean_features])
df[mode_features] = SI_mode.fit_transform(df[mode_features])

# 2. Handle ordinal encoding
df[OE_features] = OE.fit_transform(df[OE_features])

# 3. Handle one-hot encoding separately
ohe_transformed = OHE.fit_transform(df[OHE_features])
ohe_feature_names = OHE.get_feature_names_out(OHE_features)

# 4. Convert sparse matrix to DataFrame
ohe_df = pd.DataFrame(
    ohe_transformed.toarray(),
    columns=ohe_feature_names,
    index=df.index
)

# 5. Drop original OHE columns and join encoded ones
df = df.drop(columns=OHE_features)
df = pd.concat([df, ohe_df], axis=1)

In [54]:
column = df.pop("Time_taken")
df.insert(20, column.name, column)

In [ ]:
df = df.drop_duplicates()
df = df.dropna()

# Split features and target
X = df.drop(['Time_taken'], axis=1)
y = df['Time_taken']

df.to_csv("preprocessed_train.csv", index=False)

In [56]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb

# Drop any duplicates or nulls
df = df.drop_duplicates()
df = df.dropna()

# Split features and target
X = df.drop(['Time_taken'], axis=1)
y = df['Time_taken']


# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'XGBoost': xgb.XGBRegressor(random_state=42)
}

# Train and evaluate models
results = {}
for name, model in models.items():
    # Train model
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    train_pred = model.predict(X_train_scaled)
    test_pred = model.predict(X_test_scaled)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    test_r2 = r2_score(y_test, test_pred)
    
    results[name] = {
        'Train RMSE': train_rmse,
        'Test RMSE': test_rmse,
        'Test R2': test_r2
    }

# Print results
for name, metrics in results.items():
    print(f"\n{name}:")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.4f}")

# Get best model based on test RMSE
best_model_name = min(results, key=lambda x: results[x]['Test RMSE'])
print(f"\nBest Model: {best_model_name}")

# Feature importance for best model
if best_model_name in ['Random Forest', 'Gradient Boosting', 'XGBoost']:
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': models[best_model_name].feature_importances_
    })
    print("\nTop 10 Most Important Features:")
    print(feature_importance.sort_values('importance', ascending=False).head(10))


Linear Regression:
Train RMSE: 6.2085
Test RMSE: 6.2107
Test R2: 0.5601

Random Forest:
Train RMSE: 1.7623
Test RMSE: 4.7408
Test R2: 0.7437

Gradient Boosting:
Train RMSE: 4.9965
Test RMSE: 5.0296
Test R2: 0.7115

XGBoost:
Train RMSE: 3.6431
Test RMSE: 4.2335
Test R2: 0.7956

Best Model: XGBoost

Top 10 Most Important Features:
                         feature  importance
15        traffic_weather_impact    0.291270
8            multiple_deliveries    0.132708
1        Delivery_person_Ratings    0.110638
0            Delivery_person_Age    0.098922
6              Vehicle_condition    0.090715
16  vehicle_capacity_utilization    0.074777
5           Road_traffic_density    0.041083
14        avg_delivery_time_area    0.038989
4              Weatherconditions    0.035095
12                    Hour_order    0.033444


In [57]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import RobustScaler
import xgboost as xgb

robust_scaler = RobustScaler()
X_train_scaled = robust_scaler.fit_transform(X_train)
X_test_scaled = robust_scaler.transform(X_test)

xgb = xgb.XGBRegressor(random_state=42)

xgb.fit(X_train_scaled, y_train)

y_pred = xgb.predict(X_test_scaled)

print(f'R2 Score: {r2_score(y_test, y_pred):.4f}')

R2 Score: 0.7956


In [ ]:

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import RobustScaler
import numpy as np

# Drop any duplicates or nulls
df = df.drop_duplicates()
df = df.dropna()

# Split features and target
X = df.drop(['Time_taken'], axis=1)
y = df['Time_taken']

# Drop any unnecessary columns
X = X.drop(['Unnamed: 0'], axis=1) 

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def improve_model(X_train, X_test, y_train, y_test):
    # Handle outliers
    robust_scaler = RobustScaler()
    X_train_scaled = robust_scaler.fit_transform(X_train)
    X_test_scaled = robust_scaler.transform(X_test)

    # Try multiple models
    models = {
        'rf': RandomForestRegressor(n_estimators=100, random_state=42),
        'gb': GradientBoostingRegressor(random_state=42)
    }

    # Cross validation
    for name, model in models.items():
        cv_scores = cross_val_score(model, X_train_scaled, y_train, 
                                  cv=5, scoring='r2')
        print(f"{name} CV R2: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

    # Train best model (Gradient Boosting)
    best_model = GradientBoostingRegressor(n_estimators=200, 
                                         max_depth=5,
                                         learning_rate=0.1,
                                         random_state=42)
    best_model.fit(X_train_scaled, y_train)
    
    return best_model, X_train_scaled, X_test_scaled